#Realizando Select com SQL e Pyspark
by Adilson Albino

In [0]:
#Lendo arquivo de modelos de carros
df_carros = spark \
.read.format("csv")\
.option("header", True)\
.load("/aprendizado/modelo_carro")
df_carros.show(5)

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
|       5|      Escort|$77466.89|       17|
+--------+------------+---------+---------+
only showing top 5 rows



###Realizando select usando SQL

In [0]:
#Criando uma nova tabela temporária para usar de exemplo no SQL
df_carros.createOrReplaceTempView("Carros")

In [0]:
####-Para tornar essa celula capaz de ler comandos SQL é necessário usar o comando "%sql" isso server para outros comandos também.
####-Na celula que vai usar o select não deve ter nenhuma comentário, se não ele não reconchece o código SQL 

In [0]:
%sql
SELECT
  ID_CARRO,
  MODELO_CARRO
FROM
  carros

In [0]:
#Criando um dataframe com os dados retornado pelo comando SQL
df_carros_sql = spark\
.sql("""
    SELECT
        ID_CARRO AS ID,
        MODELO_CARRO AS MODELO,
        COD_MARCA AS MARCA
    FROM
        CARROS
     
     """)
df_carros_sql.show(6)

+---+----------+-----+
| ID|    MODELO|MARCA|
+---+----------+-----+
|  1|    Avalon|   54|
|  2|       RDX|    1|
|  3|      Golf|   55|
|  4|        EX|   23|
|  5|    Escort|   17|
|  6|Expedition|   17|
+---+----------+-----+
only showing top 6 rows



###Realizando select usando SPARK

In [0]:
#No caso do SPARK não precisamos criar uma tabela temporária, podemos usar o próprio dataframe já disponível com os dados
df_carros_spark = df_carros.select("ID_CARRO", "MODELO_CARRO")
df_carros_spark.show(10)

+--------+------------+
|ID_CARRO|MODELO_CARRO|
+--------+------------+
|       1|      Avalon|
|       2|         RDX|
|       3|        Golf|
|       4|          EX|
|       5|      Escort|
|       6|  Expedition|
|       7|     Voyager|
|       8|       Civic|
|       9|    Defender|
|      10|V8 Vantage S|
+--------+------------+
only showing top 10 rows



In [0]:
#Como criar ALIAS com spark?
#Para isso vamos utilizar o comando "selectExpr"
df_carros_spark = df_carros.selectExpr("ID_CARRO as ID", "MODELO_CARRO AS MODELO")
df_carros_spark.show(5)


+---+------+
| ID|MODELO|
+---+------+
|  1|Avalon|
|  2|   RDX|
|  3|  Golf|
|  4|    EX|
|  5|Escort|
+---+------+
only showing top 5 rows



In [0]:
#Segunda opção para usar apelidos no spark
#Para isso precisamos que o spark entenda o que é uma coluna, então vamos usar o camando "col().alias"
#Precisaremos também declarar o col(), para isso vamos importar as funções do pyspark
from pyspark.sql.functions import *  

df_carros_spark = df_carros.select(col("ID_CARRO").alias("ID"), col("MODELO_CARRO").alias("modelos"))
df_carros_spark.show(6)


+---+----------+
| ID|   modelos|
+---+----------+
|  1|    Avalon|
|  2|       RDX|
|  3|      Golf|
|  4|        EX|
|  5|    Escort|
|  6|Expedition|
+---+----------+
only showing top 6 rows



In [0]:
%sql
select
  *
from
  (select 
    modelo_carro,
    count(modelo_carro) as contador
  from carros
    group by
      modelo_carro) 




In [0]:

value_count = df_carros.groupBy([("id_carro"),("modelo_carro")]).count()
value_count.show(5)

+--------+------------+-----+
|id_carro|modelo_carro|count|
+--------+------------+-----+
|      33|     Mustang|    1|
|     733|         MR2|    1|
|     126|      Taurus|    1|
|     439|       Capri|    1|
|     674|         300|    1|
+--------+------------+-----+
only showing top 5 rows



In [0]:
max_value_count = value_count.orderBy(col("count").desc()).limit(1)
display(max_value_count)

modelo_carro,count
S80,8
